# Python for (open) Neuroscience

_Lecture 0.4_ - Functions / Classes and objects

Luigi Petrucco


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec-2025/blob/main/lectures/Lecture0.4_Classes-objects.ipynb)

## Lecture outline
- recap on functions
- intro to classes and objects

### Functions - recap

<p align="center">
  <img src="./files/function_anatomy.png" />
</p>

In [2]:
def print_args_and_sum(a, b, c=3):
    print(f"a={a}, b={b}, c={c}")

    return a + b + c


values_sum = print_args_and_sum(1)
print(values_sum)

TypeError: print_args_and_sum() missing 1 required positional argument: 'b'

### The scope of function variables

<span style="color:indianred">scope</span> of a variable is the technical term to indicate the parts of your code where that variable will be known to the interpreter

Variables defined inside the function will live only during the execution of the function!

In [3]:
def sum_vals(a, b):
    local_sum_variable = a + b
    return local_sum_variable



a = 1
b = 2
d = sum_vals(a, b)

a = 2
b = 3
d = sum_vals(a, b)

# this will fail, as the scope of this variable is only the function body: 
# it will live and die within the function execution:
print(local_sum_variable)

NameError: name 'local_sum_variable' is not defined

[A resource that you can use to explore scopes and variable assignments on and out from functions is [Python tutor](https://pythontutor.com/render.html#mode=display), a tool that helps you display the flow of small snippets of python code. Try it out if you want! (but don't worry if it looks confusing - it can be a bit advanced)]

On the other side, functions can have access to variables defined outside the function. However, we should not use them in the function code!

In [6]:
def sum_vals(a, b):
    # This will not fail. However, it should be avoided in general! it makes running the function
    # implicitely relying on code in the rest script.
    # This line will fail if we move the function to another notebook!
    print(a_var_not_passed)  
    return a + b


a = 1
b = 2
a_var_not_passed = [1, 2]
d = sum_vals(a, b)


[1, 2]


A function should in general be an **independent** piece of code that can be executed everywhere. If we make it dependent on variables in the script this makes it very context dependent! (you won't be able to run it in another notebook or script) 

## Side effects

**Avoid <span style="color:indianred">side effects</span>!** Do not modify the passed values or variables existing ouside the function scope. 

If you want to do it, do it consciously and make it very explicit in the name of the function or in the docstring!

In [10]:
# Here we will have troubles: the code modifies the list by popping out elements.
# A possible solution would be making a copy inside the function 
# (or write differently the code, but this is just an example)

a_list = [1, 2, 3]

def take_last(a_list):
    a_list_copy = a_list.copy()
    return a_list_copy.pop()  # this affects the original list

a = take_last(a_list)
print("Last element is: ", a)
print(a_list)

Last element is:  3
[1, 2, 3]


Sometimes you want to change input variables (eg, when you don't want 
to duplicate large datasets in memory). In those cases functions
should be very explicitly indicating this way of operating on variables.


<span style="color:indianred">inplace changes</span> is the term we use to describe modifications to arguments inside the function

In [ ]:
def double_list_values_inplace(input_list):
    """This function doubles values inside a list INPLACE.
    """
    for index, value in enumerate(input_list):
        input_list[index] = value*2
        
    # Note that I do not need to return any value here: I'm changing input_list directly!
        
str_list = [1, 10, 23]
print(str_list)
double_list_values_inplace(str_list)
print(str_list)  # the same list will contain different values now

In [ ]:
# Make sure you understand why the above function is different from the following:

def double_list_values(input_list):
    return [val * 2 for val in input_list]

# Imported functions

Many times you will import functions from external modules, like `random`. the functions docs give you all the information you need about how to use the function:

In [11]:
import random

?random.randint

(Practical 0.4.0)

## Classes and objects

Deep down the rabbit (Python) hole...

<div style="display: block; margin-left: auto; margin-right: auto; width: 50%;">
  <img src="./files/class_structure.jpeg" width="450" height="auto" />
</div>

(credits: Heather)

## Object-oriented programming (OOP)

**Disclaimer 0:** 

Objects can be confusing at first. You're entering the realm of True Programming Problems! Some concepts might feel a bit metaphysical.

**Disclaimer 1:**

This lecture is aimed more at understanding/reading existing code than at implementation. You probably won't write a lot of classes, but the concepts are very important to understand more Python code!

<span style="color:indianred">Object-oriented programming</span> is a programming paradigm based on the concept of <span style="color:indianred">objects</span>, entities that _bind together data and operations on those data_.

## An example

Let's start with an example...

Imagine the following scenario:

 - You have an experiment, containing data, info on subjects, details on experimental stimulus, etc.
 - You want to implement some functions that operate on this dataset
 

**First option**: one entity -> many variables, many functions:

In [ ]:
# A bunch of variables referring to the same experiment (not defined for real here):
imaging_data = ...
stimulus_data = ...
sampling_frequency = ...

# A bunch of functions operating on those variables (not defined for real here):
resample(imaging_data, sampling_frequency, new_frequency=...)
crop_on_stimulus(imaging_data, stimulus_data, sampling_frequency, padding=...)

But: many variables around, even though we always refer to a single entity

**Second option**: one entity -> one variable, many functions:

In [ ]:
# A dictionary with bunch of keys referring to the same experiment (not defined for real here):
data_dictionary = {"sampling_frequency": ...,
                   "imaging_data": ...,
                   "stimulus_data": ...}

# A bunch of functions operating on this dictionary (not defined for real here):
resample(data_dictionary, new_frequency=...)
crop_on_stimulus(data_dictionary, padding=...)
...

But: notice how we have some functions that always require a given variable to run (`data_dictionary`).

It could be useful to represent together data that refer to some entity, and operations that we can do on them!

## \*Enter objects\*

Objects are entities that keep together:

 - <span style="color:indianred">Attributes</span>: similar to entries in a dictionary, values of any types kept together in the same place

 - <span style="color:indianred">Methods</span>: similar to functions, but operating with and on the attributes of an object (and potentially taking additional inputs)

Advantages of object-oriented programming:

 - represent together data and procedures operating on them
 - flexible data interface ("A dictionary on steroids")
 - (advanced: nicely define what to expose and what to keep private (abstraction - more on this at the end))

## An example

Let's look at an example: an object keeping together experimental metadata info and loading functions

In [12]:
# You don't need to look at code in this cell for now! (Here we are defining the class for our object)
class ParticipantData:
    """Represent data from an experimental subject.
    
    Methods:
    =======
    
    get_data_path: ...
    
    get_exp_data: ...
    
    get_exp_metadata: ...
    
    Attributes:
    ===========
    ...
    
    """
    def __init__(self, participant_id, age, condition, base_path="/path/to/data"):
        self.participant_id = participant_id
        self.age = age
        self.condition = condition
        self.base_path = base_path
        self.experiment_name = "Experiment 1"
    
    def get_data_path(self):
        """Generates a file path for participant data based on their ID and condition."""
        return f"{self.base_path}/subject_{self.participant_id}_{self.condition}.csv"
    
    def get_exp_metadata(self):
        """Return participant matadata."""
        return {
            "id": self.participant_id,
            "age": self.age,
            "condition": self.condition,
        }
    
    def get_exp_data(self):
        """Loads data for the participant from a file."""
        data_path = self.get_data_path()
        print(f"Loading data from {data_path}...")  # We are not really loading here
        return [1, 2, 3]  # Dummy measurements

In [15]:
# Creation of a ParticipantData object: we pass relevant arguments in round brackets,
# as when calling functions. This operation is called "instantiation":
participant_a_data = ParticipantData(participant_id="A", age=28, condition="treatment")

### Attributes

In [16]:
# This object has a bunch of attributes; 
# in this case, participant_id, age, condition, experiment_name
# (Note that objects can have attributes that 
# we did not pass in the instantiation!
# in this case, experiment_name)

# We access them using the `object_name.attribute_name` syntax:
print(participant_a_data.age)

print(participant_a_data.experiment_name)

28
Experiment 1


Access attributes is conceptually quite similar to access a dictionary, just with a different syntax (`obj_name.attr_name` instead of `dict_name["key"]`)!

In [17]:
# This is a dictionary and its entry:
participant_data_dict = {"age": 28, "participant_id": "A"}
participant_data_dict["age"]

28

In [9]:
# This is an object and its attribute:
participant_data_obj = ParticipantData(participant_id="A", age=28, condition="treatment")
participant_data_obj.age

28

There are several advantages with using an object:
 - we force all objects to have some entries (_e.g._ id, age, condition)
 - we can implement methods operating on those entries (see later)

We can actually have a look at all attributes of an object with the `.__dict__` attribute (this is not something you do often, most of the times you'll know what you need from the docs):

In [18]:
# There is (most of the times) a .__dict__ attribute containing all attributes!
participant_a_data.__dict__

{'participant_id': 'A',
 'age': 28,
 'condition': 'treatment',
 'base_path': '/path/to/data',
 'experiment_name': 'Experiment 1'}

In [19]:
# Alternatively, we can use the vars() function
vars(participant_a_data)

{'participant_id': 'A',
 'age': 28,
 'condition': 'treatment',
 'base_path': '/path/to/data',
 'experiment_name': 'Experiment 1'}

### Methods

Methods are functions that are called directly from objects and operate upon their values.

You can generally read about methods in the object documentation (accessible with the usual `?` syntax)

In [20]:
?participant_a_data

In [21]:
participant_a_data.get_data_path()  # get location of subject data

'/path/to/data/subject_A_treatment.csv'

In [22]:
data = participant_a_data.get_exp_data()  # load subject data

Loading data from /path/to/data/subject_A_treatment.csv...


Many times with methods we don't need to pass arguments! They already have all the attributes of the object to operate on.

### Variables as objects

We have already introduced the concept of methods: e.g., with lists and strings:

In [23]:
a_string = "Some text"

a_string.split()  # This is a method! It operates on the text stored in `a_string`


['Some', 'text']

### Classes and objects

To use objects, we first have to define (or to import) classes.

- Classes are the blueprint/template from which we generate objects

- In the definition of a class, we write all required arguments for that class, and we define its methods

- A class stands to objects in the way the concept of a chair stands to each of the real-world chairs you see around, with their specific material, number of legs, etc.

For example, using the same "template" for subject data above (the class), we can produce different objects for each one of multiple subjects. Objects will have the same methods, and same attributes with different values:

In [ ]:
# An object of the `ParticipantData` class:
participant_a_data = ParticipantData(participant_id="A", age=28, condition="treatment")

# A different object of the same `ParticipantData` class:
participant_b_data = ParticipantData(participant_id="B", age=35, condition="control")

## Objects and types in Python

You were not aware of this, but you're already a proficient user of objects!

In fact, in Python all variables are objects of different classes (the data types). If you remember, you were calling  methods many times!

In [24]:
text = "Some text"  # this is an object of "str" class!
text.split()  # this is simply a method!

['Some', 'text']

In Python, even simpler variables are actually objects! For example, integers:

In [25]:
a_int = 1
a_int.to_bytes()  # even integers actually have methods!

b'\x01'

(Advanced: if you play around with standard types, you'll see that they do not really have attributes as we described above, only methods; this is because they are implemented in a way that keep the user away from accessing the data represented in the variable to avoid screw-ups. 

But this is something you do not need to dig into: be aware that variables are objects and forget about it!

### [optional] Inspect objects with `dir()`

We can check out attributes and methods of a class with the base Python function `dir`

In [11]:
participant_a_data = ParticipantData(participant_id="A", age=28, condition="treatment")

dir(participant_a_data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age',
 'base_path',
 'condition',
 'experiment_name',
 'get_data_path',
 'get_exp_data',
 'get_exp_metadata',
 'participant_id']

# `getattr()`

If we know the name of an attribute or a method, we can retrieve it with `getattr()`:

In [12]:
getattr(participant_a_data, "age")

28

### `callable()`

Ultimately, methods are just callable attributes!

We can verify what is callable and what is not using the `callable()` function:

In [15]:
for attr_name in ["age", "get_exp_data"]:
    attr = getattr(participant_a_data, attr_name)  # get attributes (and methods)
    is_callable = callable(attr)  # check if they are callable (if yes, they are methods!)
    print(f"`{attr_name}` attribute. Is it callable? {is_callable}")


`age` attribute. Is it callable? False
`get_exp_data` attribute. Is it callable? True
